# Processing Raw Predictions

We have generated a number of predictions and now we would like to investigate different ways of distilling multiple frame predictions into a single video prediction.

In [1]:
import os
import cv2
import sys
import numpy as np 
import pandas as pd 

from pathlib import Path
from video_utils import load_all_metadata
from sklearn.metrics import log_loss

In [2]:
raw_preds = np.load('raw_preds.npy', allow_pickle=True).item()

# Example
first_key = list(raw_preds.keys())[0]
raw_preds[first_key]

array([0.9975183 , 0.8965771 , 0.99732804, 0.925362  , 0.8753079 ,
       0.56844777, 0.82480985, 0.9596636 , 0.8657918 , 0.3335323 ,
       0.492263  , 0.09340085, 0.99842876], dtype=float32)

In [3]:
all_metadata= load_all_metadata()

## Calculate Loss

This competition's [evaluation metric](https://www.kaggle.com/c/deepfake-detection-challenge/overview/evaluation) is log loss which is given as:

$\textrm{LogLoss} = - \frac{1}{n} \sum_{i=1}^n \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)\right]$

We'll use sklearn's `log_loss()` method.

In [7]:
# Generate sample values
y_true = []
y_hat = []
for i in range(10):
    y_true.append(np.random.randint(2)) # int of 0 or 1
    y_hat.append(np.random.rand())      # float between 0 and 1

# Get loss with sklearn
log_loss(y_true, y_hat)

0.8437656923875849

## Simple Averaging

The most obvious and straightforward approach might be to simply average all of our raw predictions.

In [10]:
avg_preds = []
y_true = []

for path, preds in raw_preds.items():
    # Note that we clip values
    avg = np.mean(preds).clip(0.01, 0.99)
    avg_preds.append(avg)
    
    y = 1 if all_metadata.loc[all_metadata['fname'] == path]['label'].iloc[0] == 'FAKE' else 0
    y_true.append(y)
    
log_loss(y_true, avg_preds)

0.6478656541273575

## Take Max Value

Sometimes a single frame is enough evidence that we've encountered a deep fake. Let's try taking the max value and what happens.

In [134]:
avg_preds = []
y_true = []

for path, preds in raw_preds.items():
    # Note that we clip values
    avg = np.max(preds).clip(0.01, 0.99)
    avg_preds.append(avg)
    
    y = 1 if all_metadata.loc[all_metadata['fname'] == path]['label'].iloc[0] == 'FAKE' else 0
    y_true.append(y)
    
log_loss(y_true, avg_preds)

0.6439748840178592

Much worse!